In [1]:
import numpy as np
import pandas as pd

In [2]:
titanic = pd.read_csv("titanictrain.csv")
#titanic_test = pd.read_csv("titanictest.csv")

In [3]:
titanic.drop("PassengerId", axis = 1, inplace = True)
titanic.drop("Name", axis=1, inplace = True)
titanic.drop("Ticket", axis=1, inplace = True)
titanic.drop("Cabin", axis=1, inplace = True)
titanic.drop("Embarked", axis=1, inplace = True)


In [4]:
column = ["Pclass", "Sex","SibSp", "Parch", "Survived"]
titanic = titanic.reindex(columns=column)

In [5]:
def f(s):
    if s == "male":
        return 0
    else:
        return 1
titanic["Sex"] =titanic.Sex.apply(f)       #apply rule/function f
titanic.head()

,Pclass,Sex,SibSp,Parch,Survived
0,3,0,1,0,0
1,1,1,1,0,1
2,3,1,0,0,1
3,1,1,1,0,1
4,3,0,0,0,0


Cleaned the Data and removed all the continious column from it
The data used here is titanic dataset

In [6]:
X = titanic 
X = X.values #converting X into np array 

set the column array for us as the last column is Survived which is output and hence we dont count it in features

In [7]:
column = ["Pclass", "Sex","SibSp", "Parch"]

In [8]:
def gainRatio(X,column,index):
    y = [X[X[:,index]==k] for k in np.unique(X[:,index])] #split the data based on best filt index/features
    y=np.asarray(y)                                       #saved the multidimensional array as nparray
    n,c = np.unique(X[:,4],return_counts=True)            #for entropy of parent node
    eParent = calcEntro(n,c)
    infoCh =0
    splitInfo=0
    for i in range(y.shape[0]):
        n1 ,c1 = np.unique(y[i][:,4],return_counts=True)   
        temp = calcEntro(n1,c1)                            #entropy of each child node
        xxx = (np.sum(c1)/np.sum(c))
        infoCh = infoCh + (xxx*temp)                    #infoReq for child node
        splitInfo = splitInfo + (-1*xxx)*np.log2(xxx)    #split info for child node 
    iG = eParent - infoCh                               #information gain infoParent - infoChild
   # print("eparent",eParent,"infoG",infoG)
    if(splitInfo == 0):                                 #division by 0 check
    #    print(iG," ",splitInfo)
        return 0 
    gainRatio = (iG/splitInfo)                           #calc Gain Ratio
    return gainRatio

In [9]:
def findBestFit(X,column):
    maxx=-99
    f = -1
    for i in range(len(column)):        #iterate over all the rest columns for best feature
        t = gainRatio(X,column,i)
        if(maxx < t):
          #  print(" ")
            maxx = t
            f = i
    return f, maxx

In [10]:
def calcEntro(n,c):
    totalSum = np.sum(c)
    ans =0
    for i in range(len(n)):
        m = c[i]/totalSum
        ans = ans + (-1*m)*(np.log2(m))         #entropy formula
    return ans

In [11]:
def printInfo(X,column,level,entropy=0,gainRatio=0,x="NA"):
    if(x=="NA"):
        print("Level",level)
        n,c = np.unique(X[:,4],return_counts=True)
     #   print(len(n))
        for i in range(len(n)):
                print("Count of",n[i],c[i])
        print("Entropy",entropy)
        print("Reached leaf Node")
    else:
        print("Level",level)
        n,c = np.unique(X[:,4],return_counts=True)
    # print(len(n))
        for i in range(len(n)):
                print("Count of",n[i],c[i])
        print("Entropy",entropy)
        print("splitting Feature based on",x," with gain ratio",gainRatio)

In [12]:
def decisionTree(X,column,level=0):
    if(len(np.unique(X[:,4]))==1):    #base case if node is pure 
        printInfo(X,column,level)
        print(" ")
        return
    if(len(column)==0):             #base case if no feature left to split
        n,c = np.unique(X[:,4],return_counts=True)
        entropy = calcEntro(n,c)
        printInfo(X,column,level,entropy)
        print(" ")
        return
    index,gainRatio= findBestFit(X,column)   #find the best fit feature and its gainRatio
    n,c = np.unique(X[:,4],return_counts=True)
    entropy = calcEntro(n,c)                        
    name = column[index]
    printInfo(X,column,level,entropy,gainRatio,name)
    print(" ")
    level += 1                      #inc the level
    y = [X[X[:,index]==k] for k in np.unique(X[:,index])]     #split the data on basis of feature
    y1=np.asarray(y)
    cname = column[index]
    csize = len(column)
    del column[index]
    
    for i in range(y1.shape[0]):
        #y1 = np.delete(y1[i], np.s_[index:index+1], axis=1)
        decisionTree(y1[i],column,level)
        if(len(column) < csize):
            column.append(cname)
    return

In [13]:
column = ["Pclass", "Sex","SibSp", "Parch"]
decisionTree(X,column,0)

Level 0
Count of 0 549
Count of 1 342
Entropy 0.9607079018756469
splitting Feature based on Sex  with gain ratio 0.23249201788302346
 
Level 1
Count of 0 468
Count of 1 109
Entropy 0.6991817891208407
splitting Feature based on Pclass  with gain ratio 0.026829721319769115
 
Level 2
Count of 0 77
Count of 1 45
Entropy 0.9497869033961421
splitting Feature based on SibSp  with gain ratio 0
 
Level 3
Count of 0 77
Count of 1 45
Entropy 0.9497869033961421
splitting Feature based on Parch  with gain ratio 0
 
Level 4
Count of 0 77
Count of 1 45
Entropy 0.9497869033961421
Reached leaf Node
 
Level 2
Count of 0 91
Count of 1 17
Entropy 0.6280710155029396
splitting Feature based on Pclass  with gain ratio 0.017829416471141307
 
Level 3
Count of 0 67
Count of 1 9
Entropy 0.524805134842907
splitting Feature based on Parch  with gain ratio 0
 
Level 4
Count of 0 67
Count of 1 9
Entropy 0.524805134842907
splitting Feature based on SibSp  with gain ratio 0
 
Level 5
Count of 0 67
Count of 1 9
Entropy

In [14]:
column

['Sex', 'SibSp', 'Pclass', 'Parch']

## Changes Made

In [15]:
#best fit mai return f, maxx
#column delete krne ke baad waapis append bhi krna tha in decisiontreefunction because wrna woh further uske basis pr kabhi split nhi kr paayega for the different levels